#testing
对训练数据进行特征工程，这里我直接套了phase2的数据，所以是有label的

In [178]:
import math
from datetime import datetime,timedelta
import numpy as np
import pandas as pd
names=locals()

In [179]:
link_time={}
time_train={}
time_predict={}
time_check={}
test_time_train={}
test_time_check={}
weathers={}
rainingTotalTime={}


In [180]:
file_path='/Users/vayne/Desktop/DM_Project_0522due/data5002/phase1_test/table5.csv'

In [181]:
# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    if(time.hour in [7,16]):
        try:
            #依序解析該筆紀錄於各link所花費的時間
            for each_seq in travel_seq.split(';'):
                each_seq=each_seq.split('#')
                link_id=each_seq[0]
                start_datetime=each_seq[1]
                start_datetime=datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
                use_seconds=float(each_seq[2])
                if(time.month*31+time.day not in link_time[link_id]):
                    link_time[link_id][time.month*31+time.day]={}
                if(time.hour not in link_time[link_id][time.month*31+time.day]):
                    link_time[link_id][time.month*31+time.day][time.hour]=[]
                    
                link_time[link_id][time.month*31+time.day][time.hour].append(use_seconds)
        except:pass

In [182]:
date_key=link_time['100'].keys()

In [183]:
date_key

dict_keys([328, 329, 330, 331, 332, 333, 334])

In [184]:
date_key1=[datetime.strptime('2016-10-18', "%Y-%m-%d"),datetime.strptime('2016-10-19', "%Y-%m-%d"),\
           datetime.strptime('2016-10-20', "%Y-%m-%d"),datetime.strptime('2016-10-21', "%Y-%m-%d"),\
           datetime.strptime('2016-10-22', "%Y-%m-%d"),datetime.strptime('2016-10-23', "%Y-%m-%d"),\
           datetime.strptime('2016-10-24', "%Y-%m-%d")]

In [185]:
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
time_dict=['8_9','9_10','17_18','18_19']
for i in route_dict:
    for j in time_dict:
        names[str(i)+'_'+str(j)+'_'+'hr_avg']=[]
        for k in date_key1:
            if j in ['8_9','9_10']:
                names[str(i)+'_'+str(j)+'_'+'hr_avg'].append(['predict',i,int(k.month*31+k.day),str(k.month)+'-'+str(k.day),\
                                                        int(j[0]),k.weekday()])
                                                                                       
            if j in ['17_18','18_19']:
                names[str(i)+'_'+str(j)+'_'+'hr_avg'].append(['predict',i,int(k.month*31+k.day),str(k.month)+'-'+str(k.day),\
                                                        int(j[:2]),k.weekday()])
        
        
        
    

In [186]:
link_route1={}
link_route1['A_2']=['110','123','107','108','120','117']
link_route1['A_3']=['110','123','107','108','119','114','118','122']
link_route1['B_1']=['105','100','111','103','116','101','121','106','113']
link_route1['B_3']=['105','100','111','103','122']
link_route1['C_1']=['115','102','109','104','112','111','103','116','101','121','106','113']
link_route1['C_3']=['115','102','109','104','112','111','103','122']

In [187]:
#把前一个小时各个link的average travel time 加入list里，如果有空值，用'null'代替（早上）
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
for i in route_dict:
    temp_link=link_route1[i]
    for j in ['8_9','9_10']:
        names[str(i)+'_'+str(j)+'_'+'feature']=[]
        temp=names[str(i)+'_'+str(j)+'_'+'hr_avg']
        
        for k in temp:
            temp_row=k
            temp_date=k[2]
            
            for l in temp_link:
                if temp_date in link_time[l]:
                    if 7 in link_time[l][temp_date]:
                        temp2=np.mean(link_time[l][temp_date][7])
                    else: temp2='null'
                if temp_date not in link_time[l]: temp2='null'
            
                temp_row.append(temp2)
            names[str(i)+'_'+str(j)+'_'+'feature'].append(temp_row)

In [188]:
#把前一个小时各个link的average travel time 加入list里，如果有空值，用'null'代替（下午）
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
for i in route_dict:
    temp_link=link_route1[i]
    for j in ['17_18','18_19']:
        names[str(i)+'_'+str(j)+'_'+'feature']=[]
        temp=names[str(i)+'_'+str(j)+'_'+'hr_avg']
        
        for k in temp:
            
            temp_row=k
            temp_date=k[2]
            
            for l in temp_link:
                
                if temp_date in link_time[l]:
                    if 16 in link_time[l][temp_date]:
                        temp2=np.mean(link_time[l][temp_date][16])
                    else: temp2='null'
                if temp_date not in link_time[l]: temp2='null'
            
                temp_row.append(temp2)
            names[str(i)+'_'+str(j)+'_'+'feature'].append(temp_row)
        

In [189]:
names[str(i)+'_'+str(j)+'_'+'feature']

[['predict',
  'C_3',
  328,
  '10-18',
  18,
  1,
  12.85875,
  12.59,
  12.22625,
  29.626875,
  23.496,
  26.56725,
  3.3534999999999995,
  36.56127659574467],
 ['predict',
  'C_3',
  329,
  '10-19',
  18,
  2,
  10.215384615384615,
  11.628461538461538,
  9.226153846153847,
  28.565384615384612,
  31.07923076923077,
  32.15317073170732,
  5.451707317073172,
  39.015283018867926],
 ['predict',
  'C_3',
  330,
  '10-20',
  18,
  3,
  11.94,
  10.6175,
  9.26,
  22.66375,
  23.437333333333335,
  34.96942857142858,
  6.30057142857143,
  35.705961538461544],
 ['predict',
  'C_3',
  331,
  '10-21',
  18,
  4,
  12.232962962962961,
  11.24925925925926,
  10.324444444444445,
  30.589259259259258,
  34.63777777777778,
  28.906461538461535,
  6.494923076923077,
  37.122602739726034],
 ['predict',
  'C_3',
  332,
  '10-22',
  18,
  5,
  14.09076923076923,
  14.791923076923077,
  11.893076923076924,
  33.71846153846154,
  32.86,
  25.944000000000003,
  5.970847457627119,
  38.381509433962265],

In [190]:
#把同一条路四个小时的信息合并
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
time_dict=['8_9','9_10','17_18','18_19']

for i in route_dict:
    names[str(i)+'_'+'whole']=[]
    for j in time_dict:
        temp=names[str(i)+'_'+str(j)+'_'+'feature']
        for k in temp:
            names[str(i)+'_'+'whole'].append(k)
        

In [191]:
names[str(i)+'_'+'whole']

[['predict',
  'C_3',
  328,
  '10-18',
  8,
  1,
  9.291111111111112,
  36.812222222222225,
  10.83,
  41.146666666666675,
  27.29625,
  21.788064516129026,
  6.675483870967742,
  29.93075],
 ['predict',
  'C_3',
  329,
  '10-19',
  8,
  2,
  6.283333333333332,
  8.339999999999998,
  8.135,
  19.785,
  15.721666666666666,
  20.287333333333333,
  7.594333333333334,
  32.3629268292683],
 ['predict',
  'C_3',
  330,
  '10-20',
  8,
  3,
  10.578,
  18.863999999999997,
  69.286,
  67.678,
  27.29,
  27.31375,
  7.534583333333335,
  29.848181818181818],
 ['predict',
  'C_3',
  331,
  '10-21',
  8,
  4,
  8.99,
  14.816249999999998,
  9.66,
  24.72625,
  42.458333333333336,
  22.89095238095238,
  10.533333333333333,
  30.84973684210527],
 ['predict',
  'C_3',
  332,
  '10-22',
  8,
  5,
  11.426666666666668,
  13.003333333333336,
  12.396666666666667,
  22.243333333333336,
  16.73,
  26.895,
  4.797857142857142,
  37.986999999999995],
 ['predict',
  'C_3',
  333,
  '10-23',
  8,
  6,
  9.17

In [192]:
#convert to dataframe
import pandas as pd
for i in route_dict:
    temp_link=link_route1[i]
    feature=['label','route','calculated_day','date','hr','weekday']
    for j in temp_link:feature.append(j)
    names[str(i)+'_df']=pd.DataFrame(names[str(i)+'_'+'whole'], columns =feature) 

In [193]:
#找null value

for i in route_dict:
    print(i)
    result = names[str(i)+'_'+'df'].isin(['null'])

    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)

    print('Names of columns which contains null:', columnNames)
    
    names[str(i)+'null_element']=[]
    names[str(i)+'null_row']=[]
    # Iterate over each column and fetch the rows number where
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            names[str(i)+'null_row'].append(row)
            names[str(i)+'null_element'].append([row,col])
    print('index of rows which contains null:',set(names[str(i)+'null_row']))
    print('elements that are null:')
    print(sorted(names[str(i)+'null_element'], key = lambda x: x[0]))
   
    names[str(i)+'null_row']=list(dict.fromkeys(names[str(i)+'null_row']))
    
    print('#of rows that include null:',len(names[str(i)+'null_row']))
    print('\n')

A_2
Names of columns which contains null: []
index of rows which contains null: set()
elements that are null:
[]
#of rows that include null: 0


A_3
Names of columns which contains null: []
index of rows which contains null: set()
elements that are null:
[]
#of rows that include null: 0


B_1
Names of columns which contains null: []
index of rows which contains null: set()
elements that are null:
[]
#of rows that include null: 0


B_3
Names of columns which contains null: []
index of rows which contains null: set()
elements that are null:
[]
#of rows that include null: 0


C_1
Names of columns which contains null: []
index of rows which contains null: set()
elements that are null:
[]
#of rows that include null: 0


C_3
Names of columns which contains null: []
index of rows which contains null: set()
elements that are null:
[]
#of rows that include null: 0




In [194]:
#add weather 
file_path= '/Users/vayne/Desktop/DM_Project_0522due/data5002/weather/weather_Oct_18_Oct_24_table7.csv'
fr = open(file_path, 'r')
fr.readline()  # skip the header
vol_data = fr.readlines()

for i in range(len(vol_data)):
    each_pass = vol_data[i].replace('"', '').split(',')
    precipitation = float(each_pass[8])
    hour=int(each_pass[1])
    time_window = each_pass[0]
    time_window = datetime.strptime(time_window, "%Y-%m-%d")
    time_window_minute = int(math.floor(time_window.minute / 20) * 20)
    time = datetime(time_window.year, time_window.month, time_window.day, time_window.hour, time_window_minute, 0)
    if time.month*31+time.day not in weathers:
        weathers[time.month*31+time.day] = {}
    for h in range(3):
        weathers[time.month*31+time.day][hour+h]=precipitation
        
#2016.10.10 data missing let it be 0 anytime
weathers[320]={}
for i in range(24):
    weathers[320][i]=0

In [195]:
for i in route_dict:
    temp1=list(names[str(i)+'_df']['calculated_day'])
    temp2=list(names[str(i)+'_df']['hr'])
    weather_list=[]
    for j in range(len(temp1)):
        weather_list.append(weathers[temp1[j]][temp2[j]])
    names[str(i)+'_df']['precipitation']= weather_list

In [196]:
A_2_df

,label,route,calculated_day,date,hr,weekday,110,123,107,108,120,117,precipitation
0,predict,A_2,328,10-18,8,1,12.185476,7.369524,4.544048,5.624048,0.779545,27.560455,0.0
1,predict,A_2,329,10-19,8,2,14.875238,8.016667,6.196825,7.213016,1.181538,34.213846,1.0
2,predict,A_2,330,10-20,8,3,14.459804,8.524510,5.256471,5.876078,1.044828,32.814483,0.0
3,predict,A_2,331,10-21,8,4,13.853864,6.979302,4.736591,5.225909,0.847000,24.679500,0.5
4,predict,A_2,332,10-22,8,5,13.966500,6.150750,4.234250,4.835500,10.826316,29.805263,0.1
5,predict,A_2,333,10-23,8,6,11.421154,6.560769,3.545000,5.397692,0.939375,25.641875,0.0
6,predict,A_2,334,10-24,8,0,14.946122,8.044898,4.964792,6.169167,1.042000,34.499200,0.0
7,predict,A_2,328,10-18,9,1,12.185476,7.369524,4.544048,5.624048,0.779545,27.560455,1.8
8,predict,A_2,329,10-19,9,2,14.875238,8.016667,6.196825,7.213016,1.181538,34.213846,0.2
9,predict,A_2,330,10-20,9,3,14.459804,8.524510,5.256471,5.876078,1.044828,32.814483,0.0


In [197]:
#add the real value into table

In [198]:
#read the real value
file_path='/Users/vayne/Desktop/DM_Project_0522due/dataSet_phase2/table5.csv'
# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
    # Step 2: Create a dictionary to store travel time for each route per time window
    travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
    for i in range(len(traj_data)):
        each_traj = traj_data[i].replace('"', '').split(',')
        intersection_id = each_traj[0]
        tollgate_id = each_traj[1]

        route_id = intersection_id + '_' + tollgate_id
        if route_id not in travel_times.keys():
            travel_times[route_id] = {}

        trace_start_time = each_traj[3]
        travel_seq = each_traj[4]
        trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
        time_window_minute = math.floor(trace_start_time.minute / 20) * 20
        start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
        time=start_time_window
        tt = float(each_traj[-1])
        # 國慶不管，因道路狀態不一樣
        if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
            continue
        # 中秋節不管，因道路狀態不一樣
        if(time.month==9 and time.day in [15,16,17]):
            continue
        if start_time_window not in travel_times[route_id].keys():
            travel_times[route_id][start_time_window] = [tt]
        else:
            travel_times[route_id][start_time_window].append(tt)

In [199]:
real_value=[]
real_value_dict={}
fw = open('/Users/vayne/Desktop/dm_pro_engin/real_value.csv', 'w')
fw.writelines(','.join(['"intersection_id"', '"tollgate_id"', '"time_window"', '"avg_travel_time"']) + '\n')
for route in travel_times.keys():
    if route not in real_value_dict.keys():
        real_value_dict[route] = {}
    route_time_windows = list(travel_times[route].keys())
    route_time_windows.sort()
    for time_window_start in route_time_windows:
        time_window_end = time_window_start + timedelta(minutes=20)
        tt_set = travel_times[route][time_window_start]
        avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
        out_line =[str(route),str(time_window_start), str(avg_tt)]
        real_value.append(out_line)    
        

In [200]:
real_value

[['B_3', '2016-10-18 00:00:00', '64.69'],
 ['B_3', '2016-10-18 00:40:00', '75.37'],
 ['B_3', '2016-10-18 01:40:00', '70.14'],
 ['B_3', '2016-10-18 02:00:00', '96.03'],
 ['B_3', '2016-10-18 03:40:00', '76.88'],
 ['B_3', '2016-10-18 06:00:00', '54.75'],
 ['B_3', '2016-10-18 06:20:00', '121.02'],
 ['B_3', '2016-10-18 06:40:00', '94.04'],
 ['B_3', '2016-10-18 07:00:00', '120.6'],
 ['B_3', '2016-10-18 07:20:00', '73.4'],
 ['B_3', '2016-10-18 07:40:00', '89.91'],
 ['B_3', '2016-10-18 08:00:00', '126.4'],
 ['B_3', '2016-10-18 08:20:00', '115.52'],
 ['B_3', '2016-10-18 08:40:00', '98.86'],
 ['B_3', '2016-10-18 09:00:00', '131.15'],
 ['B_3', '2016-10-18 09:20:00', '151.95'],
 ['B_3', '2016-10-18 09:40:00', '117.69'],
 ['B_3', '2016-10-18 10:00:00', '85.85'],
 ['B_3', '2016-10-18 10:20:00', '111.97'],
 ['B_3', '2016-10-18 10:40:00', '120.19'],
 ['B_3', '2016-10-18 11:00:00', '124.38'],
 ['B_3', '2016-10-18 11:20:00', '91.1'],
 ['B_3', '2016-10-18 11:40:00', '100.28'],
 ['B_3', '2016-10-18 12:00:

In [201]:
route

'B_1'

In [202]:
real_value_dict2={}
for i in real_value:
    i[1]=datetime.strptime(i[1], "%Y-%m-%d %H:%M:%S")
    if i[1].hour in [8,9,17,18]:
        if i[0] not in real_value_dict2.keys():
            real_value_dict2[i[0]]={}
        if i[1].month*31+i[1].day not in real_value_dict2[i[0]]:
            real_value_dict2[i[0]][i[1].month*31+i[1].day]={}
        if i[1].hour not in real_value_dict2[i[0]][i[1].month*31+i[1].day]:
            real_value_dict2[i[0]][i[1].month*31+i[1].day][i[1].hour]=[]
        real_value_dict2[i[0]][i[1].month*31+i[1].day][i[1].hour].append(float(i[2]))
            
        
    

In [203]:
real_value_dict2['C_3']

{328: {8: [233.55, 155.31],
  9: [216.72, 178.4, 144.3],
  17: [189.83],
  18: [84.47, 204.89]},
 329: {8: [188.63, 175.91, 202.04],
  9: [209.15, 143.38, 168.16],
  17: [108.94, 185.13],
  18: [273.95, 123.48, 170.41]},
 330: {8: [141.54, 194.8],
  9: [148.35, 209.36],
  17: [143.44, 138.56, 173.81],
  18: [211.71]},
 331: {8: [124.56, 214.37],
  9: [106.63, 155.75, 131.44],
  17: [193.78, 254.98],
  18: [273.59, 216.57, 242.64]},
 332: {8: [196.43],
  9: [134.16, 174.1, 135.19],
  17: [199.61, 167.41, 245.12],
  18: [167.08, 224.5, 241.57]},
 333: {9: [93.53, 211.69], 17: [198.63, 200.92], 18: [218.88, 138.99]},
 334: {8: [141.63],
  9: [142.25, 169.27],
  17: [161.79, 165.87, 111.53],
  18: [109.12]}}

In [204]:
real_value_dict_avg=[]
for i in route_dict:
    for j in date_key:
        for k in [8,9,17,18]:
            if k in real_value_dict2[i][j]:
                real_value_dict_avg.append([i,j,k,np.mean(real_value_dict2[i][j][k])])
            if k not in real_value_dict2[i][j]:
                real_value_dict_avg.append([i,j,k,'null'])
            

In [205]:
real_value_dict_avg

[['A_2', 328, 8, 75.25],
 ['A_2', 328, 9, 80.07333333333334],
 ['A_2', 328, 17, 68.89666666666666],
 ['A_2', 328, 18, 66.79666666666667],
 ['A_2', 329, 8, 95.98],
 ['A_2', 329, 9, 80.10333333333334],
 ['A_2', 329, 17, 61.29666666666666],
 ['A_2', 329, 18, 67.85333333333334],
 ['A_2', 330, 8, 80.77333333333333],
 ['A_2', 330, 9, 89.44999999999999],
 ['A_2', 330, 17, 77.91],
 ['A_2', 330, 18, 60.946666666666665],
 ['A_2', 331, 8, 62.0],
 ['A_2', 331, 9, 85.5],
 ['A_2', 331, 17, 81.89],
 ['A_2', 331, 18, 76.06666666666668],
 ['A_2', 332, 8, 63.23],
 ['A_2', 332, 9, 71.76333333333334],
 ['A_2', 332, 17, 70.7],
 ['A_2', 332, 18, 64.86666666666666],
 ['A_2', 333, 8, 62.54666666666666],
 ['A_2', 333, 9, 71.48666666666666],
 ['A_2', 333, 17, 75.01333333333334],
 ['A_2', 333, 18, 64.57666666666667],
 ['A_2', 334, 8, 73.08],
 ['A_2', 334, 9, 68.96333333333332],
 ['A_2', 334, 17, 69.28333333333333],
 ['A_2', 334, 18, 63.10999999999999],
 ['A_3', 328, 8, 206.07333333333335],
 ['A_3', 328, 9, 185.4

In [206]:
real_value_dict_avg

[['A_2', 328, 8, 75.25],
 ['A_2', 328, 9, 80.07333333333334],
 ['A_2', 328, 17, 68.89666666666666],
 ['A_2', 328, 18, 66.79666666666667],
 ['A_2', 329, 8, 95.98],
 ['A_2', 329, 9, 80.10333333333334],
 ['A_2', 329, 17, 61.29666666666666],
 ['A_2', 329, 18, 67.85333333333334],
 ['A_2', 330, 8, 80.77333333333333],
 ['A_2', 330, 9, 89.44999999999999],
 ['A_2', 330, 17, 77.91],
 ['A_2', 330, 18, 60.946666666666665],
 ['A_2', 331, 8, 62.0],
 ['A_2', 331, 9, 85.5],
 ['A_2', 331, 17, 81.89],
 ['A_2', 331, 18, 76.06666666666668],
 ['A_2', 332, 8, 63.23],
 ['A_2', 332, 9, 71.76333333333334],
 ['A_2', 332, 17, 70.7],
 ['A_2', 332, 18, 64.86666666666666],
 ['A_2', 333, 8, 62.54666666666666],
 ['A_2', 333, 9, 71.48666666666666],
 ['A_2', 333, 17, 75.01333333333334],
 ['A_2', 333, 18, 64.57666666666667],
 ['A_2', 334, 8, 73.08],
 ['A_2', 334, 9, 68.96333333333332],
 ['A_2', 334, 17, 69.28333333333333],
 ['A_2', 334, 18, 63.10999999999999],
 ['A_3', 328, 8, 206.07333333333335],
 ['A_3', 328, 9, 185.4

In [207]:
for i in route_dict:
    for j in names[str(i)+'_'+'whole']:
        temp_route=j[1]
        temp_date=j[2]
        temp_hour=j[4]
        aa=[i for i,x in enumerate(real_value_dict_avg) if x[0]==temp_route and x[1]==temp_date and x[2]==temp_hour]
        j[0]=real_value_dict_avg[aa[0]][3]
    

In [208]:
names[str(i)+'_'+'whole']

[[194.43,
  'C_3',
  328,
  '10-18',
  8,
  1,
  9.291111111111112,
  36.812222222222225,
  10.83,
  41.146666666666675,
  27.29625,
  21.788064516129026,
  6.675483870967742,
  29.93075],
 [188.85999999999999,
  'C_3',
  329,
  '10-19',
  8,
  2,
  6.283333333333332,
  8.339999999999998,
  8.135,
  19.785,
  15.721666666666666,
  20.287333333333333,
  7.594333333333334,
  32.3629268292683],
 [168.17000000000002,
  'C_3',
  330,
  '10-20',
  8,
  3,
  10.578,
  18.863999999999997,
  69.286,
  67.678,
  27.29,
  27.31375,
  7.534583333333335,
  29.848181818181818],
 [169.465,
  'C_3',
  331,
  '10-21',
  8,
  4,
  8.99,
  14.816249999999998,
  9.66,
  24.72625,
  42.458333333333336,
  22.89095238095238,
  10.533333333333333,
  30.84973684210527],
 [196.43,
  'C_3',
  332,
  '10-22',
  8,
  5,
  11.426666666666668,
  13.003333333333336,
  12.396666666666667,
  22.243333333333336,
  16.73,
  26.895,
  4.797857142857142,
  37.986999999999995],
 ['null',
  'C_3',
  333,
  '10-23',
  8,
  6,

In [209]:
#convert to dataframe
import pandas as pd
for i in route_dict:
    temp_link=link_route1[i]
    feature=['label','route','calculated_day','date','hr','weekday']
    for j in temp_link:feature.append(j)
    names[str(i)+'_df']=pd.DataFrame(names[str(i)+'_'+'whole'], columns =feature) 

In [210]:
#add weather 
file_path= '/Users/vayne/Desktop/DM_Project_0522due/data5002/weather/weather_Oct_18_Oct_24_table7.csv'
fr = open(file_path, 'r')
fr.readline()  # skip the header
vol_data = fr.readlines()

for i in range(len(vol_data)):
    each_pass = vol_data[i].replace('"', '').split(',')
    precipitation = float(each_pass[8])
    hour=int(each_pass[1])
    time_window = each_pass[0]
    time_window = datetime.strptime(time_window, "%Y-%m-%d")
    time_window_minute = int(math.floor(time_window.minute / 20) * 20)
    time = datetime(time_window.year, time_window.month, time_window.day, time_window.hour, time_window_minute, 0)
    if time.month*31+time.day not in weathers:
        weathers[time.month*31+time.day] = {}
    for h in range(3):
        weathers[time.month*31+time.day][hour+h]=precipitation
        
#2016.10.10 data missing let it be 0 anytime
weathers[320]={}
for i in range(24):
    weathers[320][i]=0

for i in route_dict:
    temp1=list(names[str(i)+'_df']['calculated_day'])
    temp2=list(names[str(i)+'_df']['hr'])
    weather_list=[]
    for j in range(len(temp1)):
        weather_list.append(weathers[temp1[j]][temp2[j]])
    names[str(i)+'_df']['precipitation']= weather_list

In [211]:
names[str(i)+'_df']

,label,route,calculated_day,date,hr,weekday,115,102,109,104,112,111,103,122,precipitation
0,194.43,C_3,328,10-18,8,1,9.291111,36.812222,10.830000,41.146667,27.296250,21.788065,6.675484,29.930750,0.0
1,188.86,C_3,329,10-19,8,2,6.283333,8.340000,8.135000,19.785000,15.721667,20.287333,7.594333,32.362927,1.0
2,168.17,C_3,330,10-20,8,3,10.578000,18.864000,69.286000,67.678000,27.290000,27.313750,7.534583,29.848182,0.0
3,169.465,C_3,331,10-21,8,4,8.990000,14.816250,9.660000,24.726250,42.458333,22.890952,10.533333,30.849737,0.5
4,196.43,C_3,332,10-22,8,5,11.426667,13.003333,12.396667,22.243333,16.730000,26.895000,4.797857,37.987000,0.1
5,null,C_3,333,10-23,8,6,9.171250,9.880000,8.305000,19.181250,20.370000,21.271667,10.950000,36.488947,0.0
6,141.63,C_3,334,10-24,8,0,10.770000,11.036429,11.332857,40.065385,26.874167,25.971304,2.571739,38.217838,0.0
7,179.807,C_3,328,10-18,9,1,9.291111,36.812222,10.830000,41.146667,27.296250,21.788065,6.675484,29.930750,1.8
8,173.563,C_3,329,10-19,9,2,6.283333,8.340000,8.135000,19.785000,15.721667,20.287333,7.594333,32.362927,0.2
9,178.855,C_3,330,10-20,9,3,10.578000,18.864000,69.286000,67.678000,27.290000,27.313750,7.534583,29.848182,0.0


In [212]:
names['B_3'+'_df']

,label,route,calculated_day,date,hr,weekday,105,100,111,103,122,precipitation
0,113.593333,B_3,328,10-18,8,1,9.851304,6.174783,21.788065,6.675484,29.930750,0.0
1,111.976667,B_3,329,10-19,8,2,17.862083,6.888750,20.287333,7.594333,32.362927,1.0
2,94.890000,B_3,330,10-20,8,3,11.683684,7.097895,27.313750,7.534583,29.848182,0.0
3,95.633333,B_3,331,10-21,8,4,10.053333,6.461333,22.890952,10.533333,30.849737,0.5
4,93.423333,B_3,332,10-22,8,5,17.626364,8.784545,26.895000,4.797857,37.987000,0.1
5,94.453333,B_3,333,10-23,8,6,8.991000,6.876000,21.271667,10.950000,36.488947,0.0
6,108.876667,B_3,334,10-24,8,0,11.805455,6.816364,25.971304,2.571739,38.217838,0.0
7,133.596667,B_3,328,10-18,9,1,9.851304,6.174783,21.788065,6.675484,29.930750,1.8
8,116.496667,B_3,329,10-19,9,2,17.862083,6.888750,20.287333,7.594333,32.362927,0.2
9,93.263333,B_3,330,10-20,9,3,11.683684,7.097895,27.313750,7.534583,29.848182,0.0


In [213]:
#output the dataframe
for i in route_dict:
    names[str(i)+'_df'].to_csv(str(i)+'.csv')
